In [2]:
%run runaway_functionsv3
%matplotlib qt
# cl = Cluster('LP_1487')
# # cld = ClusterDias('Berkeley_97')
# # plot = cl.plot_cmd(isochrones=[Isochrone(cld)])
# plot = cl.plot_traceback_psr()
# show_annotation_manager(plot)


In [8]:
dias2021 = Table.read('dias2021.tsv', format='ascii.ecsv')
dias2021['logage'].min(), dias2021['logage'].max()

(6.668, 9.999)

In [ ]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('Alessi_Teutsch_5')
print(latex_text(cl))


In [12]:
cl.runaways(
    
)

Name,RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,e_v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,K,K,km / s,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
Gaia DR3 2203047987818327936,331.44191766530,60.41928497013,828.91674800,11066.8,11060.0,61.90181727275718,24.783120427304365,63.26,25.67,--,--,2203047987818327936,1.1853,0.0174,-14.690,-12.713,0.023,0.019,1.068,12.138815,0.475061,12.314358,11.839296,816.44787600,840.95831300,0.002764,0.002868,0.003791,0.006658,"331.4419176653,60.41928497013,828.916748",-12.638,0.402,-9.407,0.398,4.3845,-36.54,29.69,-13.05,40.83,262676.24215,53.52,1.621e+05,98.77,1.457e+05,39.61,--,--


In [11]:
%run runaway_functionsv3
def generate_kinematics_latex(cl):
    # Cluster part
    cluster_part = Table(cl.all)

    # Create the combined columns
    name_column = [name.replace('_', ' ') for name in cluster_part['Cluster']]
    ra_column   = [round(ra,3) for ra in cluster_part['RA_ICRS']]
    dec_column   = [round(dec,3) for dec in cluster_part['DE_ICRS']]
    distance_column = [f"${dist:.0f}\pm{e_dist:.0f}$" for dist, e_dist in zip(cluster_part['Dist'], cluster_part['e_Dist'])]
    pmRA_column = [f"${pmRA:.1f}\pm{e_pmRA:.1f}$" for pmRA, e_pmRA in zip(cluster_part['pmRA'], cluster_part['e_pmRA'])]
    pmDE_column = [f"${pmDE:.1f}\pm{e_pmDE:.1f}$" for pmDE, e_pmDE in zip(cluster_part['pmDE'], cluster_part['e_pmDE'])]
    vr_column = [f"${RV:.1f}\pm{e_RV:.1f}$" for RV, e_RV in zip(cluster_part['RV'], cluster_part['e_RV'])]


    # Add the new columns to the table
    cluster_part['Name'] = name_column
    cluster_part['RA_ICRS'] = ra_column
    cluster_part['DE_ICRS'] = dec_column
    cluster_part['d'] = Column(distance_column, unit='(pc)')
    cluster_part[r'$\mu_{\alpha}^*$'] = Column(pmRA_column, unit=u.mas/u.yr)
    cluster_part[r'$\mu_{\delta}$'] = Column(pmDE_column, unit=u.mas/u.yr)
    cluster_part[r'$v_r$'] = Column(vr_column, unit=u.km/u.s)

    cluster_part.rename_column('RA_ICRS',r'$\alpha$')
    cluster_part.rename_column('DE_ICRS',r'$\delta$')
    cluster_part  = cluster_part['Name',r'$\alpha$',r'$\delta$','d',r'$\mu_{\alpha}^*$',r'$\mu_{\delta}$',r'$v_r$']


    # Runaway part
    runaway_part = cl.runaways()
    # Create the combined columns
    name_column = [str(name) for name in runaway_part['Source']]
    ra_column   = [round(ra,3) for ra in runaway_part['RA_ICRS_1']]
    dec_column   = [round(dec,3) for dec in runaway_part['DE_ICRS_1']]
    distance_column = [f"${dist:.0f}\pm{e_dist:.0f}$" for dist, e_dist in zip(runaway_part['rgeo'], (runaway_part['B_rgeo']-runaway_part['b_rgeo']))]
    pmRA_column = [f"${pmRA:.1f}\pm{e_pmRA:.1f}$" for pmRA, e_pmRA in zip(runaway_part['pmRA'], runaway_part['e_pmRA'])]
    pmDE_column = [f"${pmDE:.1f}\pm{e_pmDE:.1f}$" for pmDE, e_pmDE in zip(runaway_part['pmDE'], runaway_part['e_pmDE'])]
    vr_column = [f"${RV:.1f}\pm{e_RV:.1f}$" for RV, e_RV in zip(runaway_part['RV'], runaway_part['e_RV'])]
    vpec2d_column = [f"${v_pec:.1f}\pm{e_v_pec:.1f}$" for v_pec, e_v_pec in zip(runaway_part['v_pec'], runaway_part['e_v_pec'])]
    # Add the new columns to the table
    runaway_part['Name'] = name_column
    runaway_part['RA_ICRS_1'] = Column(ra_column, unit='deg')
    runaway_part['DE_ICRS_1'] = Column(dec_column, unit='deg')
    runaway_part['d'] = Column(distance_column, unit='pc')
    runaway_part[r'$\mu_{\alpha}^*$'] = Column(pmRA_column, unit=u.mas/u.yr)
    runaway_part[r'$\mu_{\delta}$'] = Column(pmDE_column, unit=u.mas/u.yr)
    runaway_part[r'$v_r$'] = Column(vr_column, unit=u.km/u.s)
    runaway_part[r'$v_\text{trans}$'] = Column(vpec2d_column, unit=u.km/u.s)

    runaway_part.rename_column('RA_ICRS_1',r'$\alpha$')
    runaway_part.rename_column('DE_ICRS_1',r'$\delta$')
    runaway_part  = runaway_part['Name',r'$\alpha$',r'$\delta$','d',r'$\mu_{\alpha}^*$',r'$\mu_{\delta}$',r'$v_r$',r'$v_\text{trans}$']
    
    
    warnings.filterwarnings("ignore", category=MergeConflictWarning)
    kinematic_table=vstack([cluster_part,runaway_part])


    output = io.StringIO()
    
    latexdict_kinematics = {
    'tabletype': 'table*',
    'tablealign': 'h',
    'header_start': r'\hline',
    'data_start': r'\hline',
    'data_end': r'\hline', 
    'caption': f'Kinematics of {cl.name.replace("_"," ")} compared to its runaway'
                +r'\label{tab:' + f'{cl.name}_kinematics' + '}',
    'preamble':r'\fontsize{9pt}{11pt}\selectfont',
                
    }
    
    ascii.write(kinematic_table, output, format='latex', latexdict=latexdict_kinematics)
    latex_string = output.getvalue()
    output.close()
    return latex_string

def latex_text(cluster, n_members=10):
    cld = ClusterDias(cluster.name)
    ra, dec = cluster.skycoord.ra, cluster.skycoord.dec
    ra_str, dec_str = ra.to_string(format='latex')[1:-1], dec.to_string(format='latex')[1:-1]
    dist_str = str(round(cluster.distance.value,1)) + r"\pm" + str(round(cluster.e_distance.value,1))
    
    intro_text = rf"""\subsection{{Introduction}}
    {cluster.name.replace('_', ' ')} is located at $\alpha = {ra_str}, \delta = {dec_str}$ at a distance of ${dist_str}$ pc.
    {cluster.N} out of the {cld.N} mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{{tab:{cluster.name}_kinematics}}. Some of the brightest memebrs fo the cluster are shown in table \ref{{tab:{cluster.name}_members}}.
    
    \subsection{{CMD and proper motion}}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{{fig:{cluster.name}_cmd_pm}} in the left. The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_cmd.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The CMD showing the temperature estimate of the runaway. The isochrone for the temperature estimate is shown in blue \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_cmd_pm}}
    \end{{figure}}    

    \subsection{{Traceback}}
    
    The runaway star's motion relative to the cluster is shown in figure \ref{{fig:{cluster.name}_traceback.pdf}}.
    
    \begin{{figure}}
    \centering
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_traceback_clean.pdf}}
    \includegraphics[width=0.49\linewidth]{{Results/{cluster.name}/{cluster.name}_pm.pdf}}
    \caption{{\\ 
    \textit{{Left}}: The runaway star ...'s motion with respect to the cluster. The four green lines depict the four extreme cases of the proper motion considering the errors and the green ellipse represents the possible positions of the star 100 kyr ago. \\
    \textit{{Right}}: Proper motion diagram depicting that the star is an outlier in proper motion in the local rest frame.}}
        \label{{fig:{cluster.name}_traceback_clean&pm}}
    \end{{figure}}
    
    \subsection{{Traceback}}
    """
    
    # Members table generation
    table = cluster.mymembers
    table.sort('Gmag')
    table = table[:n_members]
    
    formatted_distances = []
    formatted_parallaxes = []
    formatted_gmags = []
    formatted_bprps = []
    
    for row in table:
        distance = row['rgeo']
        plx = row['Plx']
        e_plx = row['e_Plx']
        upper_error = row['B_rgeo'] - row['rgeo']
        lower_error = -row['b_rgeo'] + row['rgeo']
        gmag = row['Gmag']
        e_gmag = row['e_Gmag']
        bprp = row['BP-RP']
        e_bprp = row['e_BP-RP']
        
        formatted_distance = f"${distance:.0f}^{{+{upper_error:.0f}}}_{{-{lower_error:.0f}}}$"
        formatted_parallax = f"${plx:.4f}\pm{e_plx:.4f}$"
        formatted_gmag = f"${gmag:.3f}\pm{e_gmag:.3f}$"
        formatted_bprp = f"${bprp:.3f}\pm{e_bprp:.3f}$"
        
        formatted_distances.append(formatted_distance)
        formatted_parallaxes.append(formatted_parallax)
        formatted_gmags.append(formatted_gmag)
        formatted_bprps.append(formatted_bprp)

    table['formatted_distance'] = formatted_distances
    table['formatted_parallax'] = formatted_parallaxes
    table['formatted_gmag'] = formatted_gmags
    table['formatted_bprp'] = formatted_bprps
    
    t_dict_members = {
        'Gaia DR3 Source': table['Source'],
        r"$r_{\text{geo}}$ (pc)": table['formatted_distance'],
        r"$\pi$ (mas)": table['formatted_parallax'],
        r"$G$ (mag)": table['formatted_gmag'],
        r"$G_{\text{BP}}-G_{\text{RP}}$ (mag)": table['formatted_bprp']
    }
    
    latexdict_members = {
        'tabletype': 'table*',
        'tablealign': 'h',
        'header_start': r'\hline',
        'data_start': r'\hline',
        'data_end': r'\hline', 
        'caption': f'Selected members of {cluster.name.replace("_"," ")}'
                    +r'\label{tab:' + f'{cluster.name}_members' + '}',
        'preamble': r'\fontsize{12pt}{12pt}\selectfont',
    }
    
    astropy_table_members = Table(t_dict_members)

    output = io.StringIO()
    
    ascii.write(astropy_table_members, output, format='latex', latexdict=latexdict_members)
    members_table_text = output.getvalue()
    output.close()
    
    
    kinematics_table_text = generate_kinematics_latex(cluster)
    # Combine intro_text, kinematics_table_text, and members_table_text
    full_text = intro_text + "\n\n" + kinematics_table_text + "\n\n" + members_table_text
    
    
    return full_text



cl = Cluster('Alessi_Teutsch_5')
print(latex_text(cl))

\subsection{Introduction}
    Alessi Teutsch 5 is located at $\alpha = 332^\circ16{}^\prime27.84{}^{\prime\prime}, \delta = 61^\circ05{}^\prime46.32{}^{\prime\prime}$ at a distance of $869.9\pm31.0$ pc.
    59 out of the 147 mentioned in Dias have been selected as members. Using these, the kinematic parameters of the cluster have been determined. These are compared to the kinematic parameters of the runaway star in \ref{tab:Alessi_Teutsch_5_kinematics}. Some of the brightest memebrs fo the cluster are shown in table \ref{tab:Alessi_Teutsch_5_members}.
    
    \subsection{CMD and proper motion}
    The parameters of the cluster's isochrone ...
    Using these parameters, the CMD plot for the cluster is shown in figure \ref{fig:Alessi_Teutsch_5_cmd_pm} in the left. The right plot in the figure shows the proper motion diagram showing that the runaway moves with a proper motion that is different from the cluster.
    
    \begin{figure}
    \centering
    \includegraphics[width=0.49\linew

In [3]:
cl.plot_pm()
cl.plot_traceback_clean()
cl.plot_traceback_psr()

10pc image downloaded in 4.706837s
50pc image downloaded in 28.883331s


<WCSAxesSubplot: >